In [1]:
import pandas as pd

In [82]:
df= pd.read_csv('F:/PROJECTS/amazon/student_resource 3/dataset/train.csv')

In [83]:
df.head()

,image_link,group_id,entity_name,entity_value
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram


In [84]:
df.shape

(263859, 4)

In [85]:
len(df['group_id'].unique())

750

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263859 entries, 0 to 263858
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   image_link    263859 non-null  object
 1   group_id      263859 non-null  int64 
 2   entity_name   263859 non-null  object
 3   entity_value  263859 non-null  object
dtypes: int64(1), object(3)
memory usage: 8.1+ MB


In [87]:
df['entity_name'].unique()

array(['item_weight', 'item_volume', 'voltage', 'wattage',
       'maximum_weight_recommendation', 'height', 'depth', 'width'],
      dtype=object)

In [88]:
entity_counts = df['entity_name'].value_counts()
min_entity_name = entity_counts.idxmin()

# Print the entity_name and its count
print(f"Entity with minimum count: {min_entity_name}, Count: {entity_counts[min_entity_name]}")

Entity with minimum count: maximum_weight_recommendation, Count: 3263


In [89]:
df[df['group_id']==226428]

,image_link,group_id,entity_name,entity_value
205213,https://m.media-amazon.com/images/I/51wPH3Euhc...,226428,depth,25.0 inch


In [90]:
df[df['group_id']==656506]

,image_link,group_id,entity_name,entity_value
136398,https://m.media-amazon.com/images/I/61+l9t9TcP...,656506,width,82.0 centimetre
184207,https://m.media-amazon.com/images/I/51XQ96OJnX...,656506,depth,420.0 millimetre


In [91]:
df[df['group_id']==101697].count()

image_link      54
group_id        54
entity_name     54
entity_value    54
dtype: int64

In [92]:
# Define the target count
target_count = 10

# Group by 'group_id' and sample rows based on the count condition
balanced_df = df.groupby('group_id').apply(lambda x: x.sample(n=min(len(x), target_count), random_state=42))

# Reset index to clean up the DataFrame
balanced_df = balanced_df.reset_index(drop=True)

# Check the distribution to confirm the result
print(balanced_df['group_id'].value_counts())

group_id
101697    10
713791    10
704724    10
705304    10
705487    10
          ..
656506     2
997333     2
314298     2
178031     1
226428     1
Name: count, Length: 750, dtype: int64


In [93]:
balanced_df.shape

(7360, 4)

In [94]:
print(balanced_df['entity_name'].value_counts())

entity_name
width                            2058
depth                            1966
height                           1787
item_weight                      1220
voltage                            96
wattage                            91
item_volume                        80
maximum_weight_recommendation      62
Name: count, dtype: int64


In [95]:
target_count = 1000

# Find entity_names with count less than 1000
entities_below_threshold = balanced_df['entity_name'].value_counts()[balanced_df['entity_name'].value_counts() < target_count].index
print(entities_below_threshold)

Index(['voltage', 'wattage', 'item_volume', 'maximum_weight_recommendation'], dtype='object', name='entity_name')


In [96]:
for entity in entities_below_threshold:
    current_count = balanced_df[balanced_df['entity_name'] == entity].shape[0]
    rows_needed = target_count - current_count
    
    # Sample additional rows from the original df (excluding rows already in balanced_df)
    additional_rows = df[(df['entity_name'] == entity) & (~df.index.isin(balanced_df.index))].sample(n=rows_needed, replace=True, random_state=42)
    
    # Add the sampled rows to balanced_df
    balanced_df = pd.concat([balanced_df, additional_rows], ignore_index=True)

In [97]:
print(balanced_df['entity_name'].value_counts())

entity_name
width                            2058
depth                            1966
height                           1787
item_weight                      1220
maximum_weight_recommendation    1000
item_volume                      1000
wattage                          1000
voltage                          1000
Name: count, dtype: int64


In [98]:
balanced_df.shape

(11031, 4)

In [99]:
print(balanced_df['group_id'].value_counts())

group_id
752266    481
120219    402
459516    254
318770    254
271537    190
         ... 
656506      2
351545      2
997333      2
226428      1
178031      1
Name: count, Length: 750, dtype: int64


In [100]:
file_path = 'F:/PROJECTS/amazon/balanced_df.csv'

# Convert the DataFrame to a CSV file
balanced_df.to_csv(file_path, index=False)